<a href="https://colab.research.google.com/github/otkata19/competition/blob/main/Akutagawa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git

In [ ]:
!apt install aptitude

In [ ]:
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y

In [ ]:
!pip install mecab-python3

In [ ]:
!pip install fasttext

In [ ]:
!find / -name mecabrc

In [ ]:
!cp /etc/mecabrc /usr/local/etc/mecabrc

In [ ]:
#MeCab確認(python)
import sys
import MeCab
mecab = MeCab.Tagger ("-Owakati")
text = mecab.parse("すもももももももものうち ")
print(text)

In [ ]:
cd drive/MyDrive/Nishika/Akutagawa/data

In [ ]:
import re

s = "［＃「五」は中見出し］"
t = "《おおがら》あああああ《おおがら》ああああ《おおがら》ああああああ"
t = "《》いい《おおがら》ああああ《おおがら》ああああああ"
result = re.sub(r'\［.*?\］', '', s)
result_t = re.sub(r'\《.*?\》', '', t)

In [ ]:
import pandas  as pd

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train

In [ ]:
ak_num = len(df_train[df_train['author'] == 1])
print(ak_num)

In [ ]:
df_train_0 = df_train[df_train['author'] == 0].sample(n=ak_num)
df_train_1 = df_train[df_train['author'] == 1]

In [ ]:
# 縦方向に連結
df_train_cat = pd.concat([df_train_0, df_train_1], axis=0)

In [ ]:
import csv
csv.field_size_limit(1000000000)

#csv読み込み
csv_train = open("train.csv", "r", errors="", newline="" )
csv_test = open("test.csv", "r", errors="", newline="" )
csv_train_data = csv.reader(csv_train, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
csv_test_data = csv.reader(csv_test, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)

In [ ]:
import re

mecab = MeCab.Tagger ("-Owakati") #分かち書き設定

def data2list(dataframe):
    list_row = [] #分かち書き結果を保存
    for row in dataframe.itertuples():
        text_r = row[2]
        text_s = re.sub(r"\s", "", text_r)
        # ［］とその中身を削除
        text_s = re.sub(r'\［.*?\］', '', text_s)
        # 《》とその中身を削除
        text_s = re.sub(r'\《.*?\》', '', text_s)
        # MeCabで形態素解析
        text = mecab.parse(text_s)
        list_row.append((str(row[3])+'   '+text).rstrip("\n"))
    return list_row

def test2list(dataframe):
    list_row = [] #分かち書き結果を保存
    for row in dataframe.itertuples():
        text_r = row[2]
        text_s = re.sub(r"\s", "", text_r)
        # ［］とその中身を削除
        text_s = re.sub(r'\［.*?\］', '', text_s)
        # 《》とその中身を削除
        text_s = re.sub(r'\《.*?\》', '', text_s)
        # MeCabで形態素解析
        text = mecab.parse(text_s)
        list_row.append((text).rstrip("\n"))
    return list_row

In [ ]:
train_pre = data2list(df_train_cat)
test_pre = test2list(df_test)

In [ ]:
test_pre[1]

In [ ]:
from sklearn.model_selection import train_test_split

train, dev = train_test_split(train_pre, random_state=0, test_size=0.1)

In [ ]:
print(len(train), len(dev), len(test_pre))

In [ ]:
#学習データ作成
path_w = 'train_data.txt' #分かち書き済み学習用データ保存ファイル名
with open(path_w, 'w') as f_out:
    for row in train:
        label, text = row.strip().split('   ')
        f_out.write('__label__{} {}\n'.format(label, text))

In [ ]:
#検証データ作成
path_w = 'dev_data.txt' #分かち書き済み学習用データ保存ファイル名
with open(path_w, 'w') as f_out:
    for row in dev:
        label, text = row.strip().split('   ')
        f_out.write('__label__{} {}\n'.format(label, text))

In [ ]:
# fastTextで学習
import fasttext as ft

#model = ft.train_supervised("train_data.txt", label_prefix='__label__',epoch=30, loss="softmax")
model = ft.train_supervised(input="train_data.txt", autotuneValidationFile="dev_data.txt")
model.save_model("akutagawa.bin")

In [ ]:
classifier = ft.load_model('akutagawa.bin')
ret = classifier.test("dev_data.txt")
print(ret)

In [ ]:
import pandas as pd

df = pd.read_csv('sample_submission.csv')
df_w = df['writing_id']
print(df_w)

In [ ]:
# test[0]は'body'なのでtest_pre[1:]
classifier = ft.load_model('akutagawa.bin')
ret_list = []
for test_text in test_pre:
    ret = classifier.predict(test_text)
    ret_list.append(ret[0][0].replace("__label__", ""))

In [ ]:
df_a = pd.DataFrame({'author': ret_list})
df_wa = pd.concat([df_w, df_a], axis=1)
print(df_wa)

In [ ]:
df_wa.to_csv('submission.csv', index=False)